<a href="https://colab.research.google.com/github/JasperLS/data_science_intros/blob/main/MNIST_and_Fashion_MNIST_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MNIST & Fashion MNIST Classification Notebook
This notebook is losely based on the example provided at https://victorzhou.com/blog/keras-neural-network-tutorial/: 

MNIST is publicly available image data set which consists of 28 by 28 pixels image of hand-written single digits. So each image in MNIST is an image of a digit from 0 to 9. MNIST challenge is to develop a machine learning algorithm that can classify these images into 10 classes (0 to 9).

Fashion-MNIST is a dataset of Zalando's article images—consisting of a training set of 60,000 examples and a test set of 10,000 examples. Each example is a 28x28 grayscale image, associated with a label from 10 classes. 

## Setup

In [ ]:
!pip install mnist

In [ ]:
import numpy as np
import mnist
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Conv2D,MaxPooling2D,Flatten
from tensorflow.keras.utils import to_categorical

In [ ]:
train_images = mnist.train_images()
train_labels = mnist.train_labels()
test_images = mnist.test_images()
test_labels = mnist.test_labels()

In [ ]:
# load alternative fashion mnist dataset
# fashion_mnist = tf.keras.datasets.fashion_mnist
# (train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

## Understand the Data

In [ ]:
# how many examples do we have?
print(len(train_images))
print(len(train_labels))

In [ ]:
# what resolution do our images have?
train_images.shape

In [ ]:
# let's look at some examples
from matplotlib import pyplot as plt
num = 6123
print('Label:',train_labels[num])
plt.imshow(train_images[num])
plt.show()

In [ ]:
# how many do we have of each example?
np.unique(train_labels, return_counts=True)

In [ ]:
# how is our image data represented?
np.unique(train_images)

## Data Preparation

In [ ]:
# normalize the images
train_images = (train_images-train_images.mean())/train_images.std()
test_images = (test_images-train_images.mean())/train_images.std()

In [ ]:
# flatten the images
train_images = train_images.reshape((-1, 28*28))
test_images = test_images.reshape((-1, 28*28))

In [ ]:
# do we need to transform labels (later)?
print(to_categorical(train_labels)[:2])
(train_labels[:2])

## Get the Model

In [ ]:
# build the model
model = Sequential([
  Dense(64, activation='relu', input_shape=(784,)),
  Dense(64, activation='relu'),
  Dense(10, activation='softmax'),
])

# compile the model
model.compile(
  optimizer='adam',
  loss='categorical_crossentropy', 
  metrics=['accuracy'],
)

# let's take a look at our model
model.summary()

# why do we get the specific number of parameters?

## Train the Model

In [ ]:
model.fit(
  train_images,
  to_categorical(train_labels),
  epochs=5,
  batch_size=64,
  validation_data=(test_images, to_categorical(test_labels)),
)

## Evaluate Model

In [ ]:
model.evaluate( 
  test_images, 
  to_categorical(test_labels)
)

# predict on the first 5 test images.
predictions = model.predict(test_images[:5])

# print our model's predictions
print(np.argmax(predictions, axis=1)) 

# check our predictions against the ground truths
print(test_labels[:5]) 

## Try Alternative Model incl. CNNs

##### Get Model inlc. CNN

In [ ]:
model = Sequential([
  Conv2D(64, kernel_size=(3,3), activation='relu', input_shape=(28,28,1)),
  MaxPooling2D((2,2)),
  Flatten(),
  Dense(64, activation='relu'),
  Dense(64, activation='relu'),
  Dense(10, activation='softmax'),
])

# compile the model
model.compile(
  optimizer='adam',
  loss='categorical_crossentropy', 
  metrics=['accuracy'],
)

# let's take a look at our model
model.summary()

# why do we get the specific number of parameters?


##### Train CNN Model

In [ ]:
model.fit(
  train_images.reshape(train_images.shape[0],28,28,1),
  to_categorical(train_labels),
  epochs=5,
  batch_size=64,
  validation_data=(test_images.reshape(test_images.shape[0],28,28,1), to_categorical(test_labels)),
)

##### Evaluate CNN Model

In [ ]:
model.evaluate( 
  test_images, 
  to_categorical(test_labels)
)

# predict on the first 5 test images.
predictions = model.predict(test_images[:5])

# print our model's predictions
print(np.argmax(predictions, axis=1)) # [7, 2, 1, 0, 4]

# check our predictions against the ground truths.
print(test_labels[:5]) # [7, 2, 1, 0, 4]